Описание
Исходный репозиторий: https://github.com/srbhklkrn/depression-therapist-chatbot

Логика работы:



Предварительные действия


Ограничения:
* в репозитории есть ссылка на готовую модель и код для обучения модели
* в репозитории нет ссылки на датасет, но указано, что кагл твиттеры 1 600 00. В кагле есть подошедший https://www.kaggle.com/kazanova/sentiment140?select=training.1600000.processed.noemoticon.csv . Второй датасет https://www.kaggle.com/c/tweet-sentiment-extraction не подходит по структуре.

Описание модели
Особенности
* в репозитории есть ссылка на готовую модель и код для обучения модели, в репозитории пустышки с названием хеш суммой
* в репозитории нет ссылки на датасет, но указано, что кагл твиттеры. Вместо них пустышки с названием хеш суммой. В кагле есть основной датасет https://www.kaggle.com/c/tweet-sentiment-extraction
* обучение на датасете твиттеров, которые разбиваются на набор слов не более 20ти символов длиной, чистятся от хештегов и авторов, ссылок и слов с xxx. 
* разбитые отрезки токенизируются и чистятся от слов короче 2х символов
* словарь слов сохраняется в отдельный файл
* модель обучается на сети с одним слоем из 128 нейронов (тензорфлоу). Точность определения тональности до 82%.

Логика работы диалоговой системы:
* жесткое дерево вопросов и ответов.
* сначала выясняется общий настрой на диалог, если он негативный идет переход к выяснению чувств
* в финале большинства диалогов переход к вопросам о друзьях, родителях и работе(школе)
* после этого расказ о техники исследования своих текущих мыслей (общий кейс) или отношения к предетам друзей, родителей, работы
* логика переходов основнывается на вероятностях оценки позитивной (согласительной) тональности. Коэффициент по-видимому оподобраны опытным путем.


In [1]:
import os
from __future__ import print_function
from gensim.parsing.preprocessing import strip_non_alphanum, preprocess_string
from gensim.corpora.dictionary import Dictionary
from tensorflow import keras
from keras.models import load_model
import numpy as np
import subprocess
import time

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [2]:
# Загружаем модель
!gdown --id 1RtM-2yxJyXe9rLI62nqxHkG7by-QHKvj
!gdown --id 1DXiLwr2LrRUi-BgidxTvVLsFzoLht9i3
model = load_model('./model_nn.hdf5')
vocab = Dictionary.load('./vocab_sentiment')


Downloading...
From: https://drive.google.com/uc?id=1RtM-2yxJyXe9rLI62nqxHkG7by-QHKvj
To: /content/model_nn.hdf5
89.3MB [00:02, 40.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DXiLwr2LrRUi-BgidxTvVLsFzoLht9i3
To: /content/vocab_sentiment
6.56MB [00:00, 24.8MB/s]


In [3]:
# Функции с текстами
def predict(text):
    preprocessed = [word[:-3] if word[-3:] == 'xxx' else word for word in
                    preprocess_string(text.replace('not', 'notxxx'))]
    text_col = [(vocab.token2id[word] + 1) for word in preprocessed
                if word in vocab.token2id.keys()]
    text_col = [text_col]
    tweet_l = 20
    if len(text_col[0]) < tweet_l:
        for i in range(tweet_l - len(text_col[0])):
            text_col[0].append(0)
    elif len(text_col[0]) > tweet_l:
        while len(text_col[-1]) > tweet_l:
            text_col.append(text_col[-1][tweet_l:])
            text_col[-2] = text_col[-2][:tweet_l]
    prediction = 0
    for txt in text_col:
        prediction += model.predict(np.array([txt]), batch_size=1)
    prediction /= len(text_col)
    return prediction

# ending = 'It was extremely decent conversing with you and I trust that now you'\
#            'feel better subsequent to conversing with me.\nBest of fortunes for your future'\
#            'Bye!'

ending = 'Беседа с вами была чрезвычайно интересной, и я надеюсь, что теперь вы чувствуете себя лучше после беседы со мной. Удачи вам. Пока!'

def frnd():
    # reply = input('How are your companions getting together with your desires?\n')
    reply = input('Как ваши товарищи относятся к вашим желаниям?')
    if(predict(reply) >= 0.4): #1
        # reply = input('Did you break up recently?\n')
        reply = input('Вы недавно расстались?\n')
        if(predict(reply) >= 0.4): #2
            # print(name + ", try not to feel tragic. Take as much time as is needed and mend properly,"\
            #             "take a gander at what's happened, gain from it, and discover approaches to "\
            #             "assemble another and sound life.\nAll any of us needs is to "\
            #             "be upbeat. For a few, this requires the ideal individual to "\
            #             "be our other half, and for others, it implies finishing "\
            #             "the condition yourself. In any case, to locate the right "\
            #             "individual, you should be the ideal individual. What's more, assume that "\
            #             "over the long haul, your endeavors will prompt your own "\
            #             "individual cheerful completion.")
            print(name + ", постарайтесь не чувствовать себя трагично. Потратьте столько времени, сколько необходимо, и исправьтесь должным образом,"\
                        "взгляните на то, что произошло, извлеките из этого пользу и найдите подходы к созданию другой и здоровой жизни.\n"\
                        "Все, что нам нужно, - это быть оптимистичными. Для некоторых это требует, чтобы идеальным человеком был наш второй половинкой,"\
                        " а для других это означает, что вы сами закончите состояние. В любом случае, чтобы найти подходящего человека, вы должны быть "\
                        "идеальным человеком. Более того, предположите, что в долгосрочной перспективе ваши усилия приведут к вашему собственному жизнерадостному завершению.")
            print(ending)
        else:
            # print(name + ", try not to stress. You might be at a point where comparable "\
            #             "individuals are not in your life at this moment. That occurs in "\
            #             "life from time to time.\nIt is smarter to be far from "\
            #             "incongruent individuals and those individuals are pulled in to "\
            #             "you when you put on a show to be somebody you aren't.\nBe as "\
            #             "diverse as you really seem to be, become more acquainted with yourself at a "\
            #             "profound level, regard your uniqueness, connect with "\
            #             "individuals truly, and in the long run the general population who acknowledge "\
            #             "you will see and be attracted.")
            print(name + "постарайтесь не напрягаться. Возможно, вы находитесь в точке, где в данный момент в вашей жизни нет сопоставимых людей. Это происходит в жизни время от времени."\
                         "Разумнее быть вдали от несогласованных личностей, и эти личности притягиваются к вам, когда вы устраиваете шоу, чтобы быть кем-то, кем вы не являетесь.\n"\
                         "Каким бы разнообразным вы ни казались на самом деле, лучше познакомьтесь с собой на глубоком уровне, оцените свою уникальность, по-настоящему общайтесь с людьми, "\
                         "и в долгосрочной перспективе все окружение, которое знает вас, оценит и привяжется к вам")

            print(ending)
    else:
        # print("Many individuals have a tendency to expect excessively of others, their family, "\
        #       "their companions or even just colleagues. It's a typical mistake"\
        #       ", individuals don't think precisely the way you do.\nDon't let the "\
        #       "suppositions of others influence you to overlook what you merit. You are "\
        #       "not in this world to satisfy the desires of others, "\
        #       "nor should you feel that others are here to satisfy yours."\
        #       "\nThe initial step you should take in the event that you need to figure out how to "\
        #       "quit expecting excessively from individuals is to just acknowledge and "\
        #       "acknowledge the way that no one is flawless and that everybody "\
        #       "commits errors once in a while.")
        print("Многие люди склонны к чрезмерным ожиданиям от других, своей семьи, своих товарищей или даже просто коллег. Это типичная ошибка, люди думают не совсем так, как вы.\n"\
               "Не позволяйте предположениям других влиять на вас, чтобы упустить то, что вы заслуживаете. Вы находитесь в этом мире не для того, чтобы удовлетворять желания других,"\
               "и вы не должны чувствовать, что другие здесь для того, чтобы удовлетворять ваши потребности.\n"\
               "Первый шаг, который вы должны предпринять в том случае, если вам нужно выяснить, как перестать чрезмерно ожидать от людей, - это просто признать и признать, что никто не безупречен и что все время от времени совершают ошибки.")
        print(ending)
 
       
def family():
    # print(name + ", try not to take excessively pressure. You should simply change "\
    #             "your needs. Try not to go up against pointless obligations and "\
    #             "responsibilities.\nTake counsel from individuals whose feeling you "\
    #             "trust, and get particular counsel when issues arise.\nYou should "\
    #             "utilize pressure administration strategies and dependably seek after the best. "\
    #             "These circumstances emerge in everybody's life and what is important the "\
    #             "most is taking the correct choice at such minutes.")
    print(name + "старайтесь не испытывать чрезмерного давления. Вы должны просто изменить свои потребности. Старайтесь не идти против бессмысленных обязательств и ответственности.\n"\
                "Доверьтесь совету от людей, чьим чувствам вы доверяете, и получите конкретный совет, когда возникнут проблемы.\n"\
                "Вы должны использовать стратегии управления давлением и надежно стремиться к лучшему. Эти обстоятельства возникают в жизни каждого человека, и самое главное-сделать правильный выбор в такие минуты.")
    print(ending)

def work():
    # print(name + ", try not to take excessively pressure. I can show some extremely cool "
    #               "approaches to deal with it.\nYou ought to create sound reactions which "
    #               "incorporate doing standard exercise and taking great quality rest. "
    #               "You ought to have clear limits between your work or scholastic "
    #               "life and home life so you ensure that you don't blend them.\n"
    #               "Techniques, for example, contemplation and profound breathing activities can be "
    #               "truly helping in mitigating stress.\n Always set aside opportunity to "
    #               "revive to dodge the negative impacts of endless pressure "
    #               "what's more, burnout. We require time to recharge and come back to our pre-"
    #               "feeling of anxiety of working.")
    print(name + "старайтесь не испытывать чрезмерного давления. Я могу показать несколько чрезвычайно крутых подходов к решению этой проблемы.\n"\
          "Вы должны проговаривать реакцию и выполнять стандартные упражнений и качественно отдыхать. У вас должны быть четкие границы между вашей работой или школьной жизнью и домашней жизнью, чтобы вы не смешивали их.\n"\
          "Техники, например, созерцание и глубокое дыхание, могут действительно помочь в смягчении стресса.\n"\
          "Всегда оставляйте возможность отказаться от обязательств, чтобы избежать негативных последствий бесконечного давления, более того, выгорания. "\
          "Нам нужно время, чтобы перезарядиться и вернуться к нашему предчувствию тревоги от работы.")
    print(ending)


def sorrow1():
    # reply = input('I get it. Appears as though something\'s annoying you.'\
    #                  'Might you be able to additionally portray it, in short?\n')
    reply = input('Я все понял. Похоже, вас что-то раздражает. Не могли бы вы дополнительно кратко описать его?\n')
    if(predict(reply) >= 0.4):
        # reply = input('It appears like however, the issue may be a bit '\
        #                  'troubling, it may not really be intense. '\
        #                  'What are your musings on this?\n')
        reply = input('Похоже, что хотя проблема и тревожит вас, но она не очень сильная. Что вы думаете по этому поводу?\n')

        if(predict(reply) >= 0.5):
            # reply = input('It would appear that you concur with me. Wanna sign off?\n')
            reply = input('Похоже, вы со мной согласны. Хочешь закончить?\n')
            if(predict(reply)>0.55):
                # print("That is alright. It was pleasant conversing with you. You can talk "\
                #       "with me whenever you want.\nBye " + name + "!")
                print(f'Все в порядке. Было приятно побеседовать с вами. Ты можешь говорить со мной, когда захочешь.\n Пока {name}!')
            else:
                sorrow3() 
        else:
            sorrow3()
    else:
        sorrow2()

def sorrow2():
    # reply = input('It would be ideal if you dont hesitate to share your emotions ' + name +\
    #                  ', consider me your friend.\n')
    reply = input('Было бы идеально, если бы вы не стеснялись делиться своими эмоциями ' + name + '. Считайте, что мы друзья.\n')

    if(predict(reply)>=0.3):
        # reply = input('I see. Among the musings happening in your psyche, '\
        #                  'which one miracles you the most?\n')
        reply = input('Понимаю. Среди ваших текущих мыслей, какая из них удивляет вас больше всего?\n')
        # reply = input('For what reason do you think it upsets you?\n')
        reply = input('Как вы думаете, почему это расстраивает вас?\n')
        # print("Approve. You simply distinguished what we call a programmed thought. "\
        #       "Everybody has them. They are contemplations that instantly fly to "\
        #       "mind with no exertion on your part.\nMost of the time the "\
        #       "thought happens so rapidly you don't see it, however it has a "\
        #       "affect on your feelings. It's normally the feeling that you "\
        #       "see, instead of the thought.\nOften these programmed "\
        #       "considerations are contorted somehow yet we for the most part don't stop "\
        #       "to scrutinize the legitimacy of the idea. In any case, today, that is "\
        #       "what we will do.")
        print("Одобряю. Вы просто выделили то, что мы называем запрограммированной мыслью. Они есть у всех. Это размышления, которые мгновенно приходят в голову без каких-либо усилий с вашей стороны."\
              "Большую часть времени мысль происходит так быстро, что вы ее не видите, однако она влияет на ваши чувства. Обычно это чувство, которое вы видите, а не мысль.\n"\
              "Часто эти запрограммированные соображения каким-то образом искажены, но мы по большей части не останавливаемся, чтобы тщательно изучить основания этой идеи."\
              "Но сегодня во всяком случае, мы именно это и сделаем.")
        # reply = input('So, ' + name + ', are there signs that opposite '\
        #                  'could be true?\n')
        reply = input('Так что, ' + name + ', есть ли признаки того, что обратное может быть правдой?\n')


        if(predict(reply) >= 0.4): #3
            # print("I'm happy that you understood that the inverse could be "\
            #       "genuine. The reason these are called 'false convictions' is "\
            #       "since they are extraordinary methods for seeing the world. "\
            #       "They are dark or white and disregard the shades of dim in "\
            #       "between.\nNow that you have found out about this cool "\
            #       "strategy, you can apply it on a large portion of the issues that "\
            #       "you will confront. On the off chance that despite everything you feel stuck anytime, you "\
            #       "can simply visit with me.\nBest of fortunes for your future "\
            #       "attempts. Bye!")
            print('Я рад, что вы поняли, что обратное может быть верным. Причина, по которой они называются "ложными убеждениями", заключается в том, что они являются экстраординарными методами видения мира.'\
                  'Они темные или белые и игнорируют оттенки серого между ними. Теперь, когда вы узнали об этой классной стратегии, вы можете применить ее к значительной части проблем, с которыми вы столкнетесь.\n'\
                  'На тот случай, если, несмотря ни на что, вы в любое время почувствуете себя застрявшим, вы можете просто навестить меня.\n'\
                  'Удачи вам в ваших будущих попытках. Пока!')
        else:
            sorrow4()
    else:
        sorrow4()

def sorrow3():
    # reply = input('Feel agreeable. Would you be able to quickly clarify about your '\
    #                  'day?\n')
    reply = input('Чувствуйте себя уютно. Не могли бы вы быстро рассказать о своем дне?\n')
    # reply = input('What are the exercises that make up your a large portion of the '\
    #                  'day?\n')
    reply = input('Какие занятия составляют большую часть вашего дня?\n')
    # reply = input('It would appear that you may feel great talking '\
    #                  'about yourself. Might you be able to share your feelings?\n')
    reply = input('Кажется, вам может быть приятно говорить о себе. Не могли бы вы поделиться своими чувствами?\n')
    if(predict(reply)>=0.3):
        sorrow2()
    else:
        sorrow4()
    
def sorrow4():
    # print("My sensitivities. It would appear that it may be a state of concern. Don't "\
    #       "stress, that is what I'm here for!")
    print("Я сочуствую. Кажется, это может быть состояние беспокойства. Не переживайте, это то, для чего я здесь!")
    # reply_frnd = input('How are things going ahead with your friends?\n')
    reply_frnd = input('Как у вас дела с друзьями?\n')
    # reply_family  = input('How is your association with your parents?\n')
    reply_family  = input('Как у вас отношения с вашими родителями?\n')
    # reply_worklife = input('How is your professional or scholastic life going on?\n')
    reply_worklife = input('Как у вас продвигаются дела на работе или в школе?\n')
    if(predict(reply_frnd)<=0.3):
        frnd()
    else:
        if(predict(reply_family)<=0.3):
            family()
        else:
            work()


In [4]:
# print('\n\nHi! A debt of gratitude is in order for coming here. I am a chatbot. Individuals say that''I am a kind and receptive bot.')
print('\n\nПривет! Я благодарен, что вы пришли. Я чатбот. Многие говоря я очень добрый и отзывчивый бот!')
# name = input('If its not too much trouble let me know your name.\n')
name = input('Если вас не затруднит, скажите мне ваше имя.\n')
try:
    preprocessed = [word for word in preprocess_string(name) if word not in ('люди', 'звонок', 'друг')][0]
                    # 'people', 'call', 'friend')][0]
    name = [word for word in strip_non_alphanum(name.lower()).split() if preprocessed in word][0]
except:
    name = name.split()[0]
name = name[0].upper() + name[1:]
# print("Hi " + name + "! My name's SYVBot. How about we begin with our session.")
print("Привет " + name + "! Меня зовут SYVBot. Давай начнем нашу сессию?")
# reply = input("Hows life?\n")
reply = input("Как жизнь?\n")
if (predict(reply) >= 0.55):
    reply = input('Это отлично. А вы сами справляетесь со всем или вс еже есть пара стрессовых моментов о которых вы хотели бы поговорить?\n') 
    # reply = input('That is great. Are you as a rule this glad, or are there '\
    #                  'a few stresses that you need to talk about?\n') 
    if (predict(reply)>=0.7):
        # reply = input('You appear to be extremely content. Wanna sign off?\n')
        reply = input('Кажется вы отлично справляетесь, хотите завершить?\n')
        if(predict(reply)>=0.7):
            # print('Ok, bye ' + name + '!')
            print('Хорошо, пока' + name + '!')
        else:
            # reply = input('Is there something annoying you? Would you '\
            #                  'share it with me?\n') 
            reply = input('Есть что нибудь, что раздражает вас? Может поделитесь со мной?\n') 
            if(predict(reply)>=0.7):
                # print("That is alright. It was pleasant conversing with you. You can talk "\
                #       "with me whenever you want.\n Bye" + name + "!")
                print("Тогда отлично. Было здорово поговорить с тобой. Ты можешь поговорить со мной снова, когда захочешь. Пока " + name + "!")
            else:
                sorrow1()
    else:
        sorrow1()
else:
    sorrow3()




Привет! Я благодарен, что вы пришли. Я чатбот. Многие говоря я очень добрый и отзывчивый бот!
Если вас не затруднит, скажите мне ваше имя.
Андрей
Привет Андрей! Меня зовут SYVBot. Давай начнем нашу сессию?
Как жизнь?
Ничего так. А ты?


KeyboardInterrupt: ignored